<a href="https://colab.research.google.com/github/gsukisubramaniam/Projects/blob/master/NSE_DayBhavCopyAddition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive/')


Mounted at /content/drive/


In [2]:
import pandas as pd
import numpy as np
import datetime
from datetime import date
import os

In [3]:
# To ignore the warnings for pandas operations
pd.options.mode.chained_assignment = None

RSI calculation for all the days

In [4]:
def rsi_all(df,period=14):
    
    df.sort_values(by=['TIMESTAMP'],inplace=True)
    df.reset_index(drop=True, inplace=True)
    df['UM'] = df['CLOSE'].diff(periods=1)
    df['DM'] = df['UM']*(-1)
    df['UM'] = df['UM'].apply(lambda x: max(0,x))
    df['DM'] = df['DM'].apply(lambda x: max(0,x))
    df.sort_values(by = ['TIMESTAMP'],inplace=True)
 
    # Simple Moving Average
    df['UM_SMA'] = df.loc[:,'UM'].rolling(window=period).mean()
    df['DM_SMA'] = df.loc[:,'DM'].rolling(window=period).mean()
 
    # Exponential Moving Average
    df['UM_EMA'] = df.loc[:,'UM'].ewm(com = period-1, min_periods=period).mean()
    df['DM_EMA'] = df.loc[:,'DM'].ewm(com = period-1, min_periods=period).mean()
 
    # Relative Strength
    df['RS_SMA'] = df['UM_SMA']/df['DM_SMA']
    df['RS_EMA'] = abs(df['UM_EMA']/df['DM_EMA'])
 
    # Relative Strength Index (RSI)
    df['RSI_SMA'] = round(df['RS_SMA'].apply(lambda x: 100-(100/(x+1))),2)
    df['RSI_EMA'] = round(df['RS_EMA'].apply(lambda x: 100-(100/(x+1))),2)
    df.sort_values(by = ['TIMESTAMP'],inplace=True)
    df.reset_index(drop=True, inplace=True)
    #     return df
    output_df = df
    return output_df

Utility Function to calculate the RSI for the latest date.

In [5]:
def group_df(df, interval='W'):
    df['TIMESTAMP'] = pd.to_datetime(df['TIMESTAMP'])
    df.reset_index(drop = True, inplace=True)
    # Getting year, month, & weeknum. Weeknum is common across years to we need to create unique index by using year and weeknum
    df['Year'] = df['TIMESTAMP'].dt.year
        # Getting month number
    df['Month_Number'] = df['TIMESTAMP'].dt.month
    # Getting week number
    df['Week_Number'] = df['TIMESTAMP'].dt.week
    if (interval == 'W'):
        df_g = df.groupby(['Year','Week_Number']).agg({'SYMBOL':'last','SERIES':'last','OPEN':'first', 'HIGH':'max', 'LOW':'min', 'CLOSE':'last', 'TIMESTAMP':'last','RS_SMA':'last', 'DM_EMA':'last', 'RSI_EMA':'last', 'UM_EMA':'last', 'UM_SMA':'last', 'RSI_SMA':'last', 'RS_EMA':'last', 'UM':'last', 'DM':'last', 'DM_SMA':'last'})
        df_g.reset_index(drop=True,inplace=True)
    elif (interval == 'M'):
        df_g = df.groupby(['Year','Month_Number']).agg({'SYMBOL':'last','SERIES':'last','OPEN':'first', 'HIGH':'max', 'LOW':'min', 'CLOSE':'last', 'TIMESTAMP':'last','RS_SMA':'last', 'DM_EMA':'last', 'RSI_EMA':'last', 'UM_EMA':'last', 'UM_SMA':'last', 'RSI_SMA':'last', 'RS_EMA':'last', 'UM':'last', 'DM':'last', 'DM_SMA':'last'})
        df_g.reset_index(drop=True,inplace=True)
    else: 
        print ('Invalid Inputs')
    return df_g

In [6]:
def rsi(df, time_interval = 'D',period=14):
    cols_seq = ['SYMBOL', 'SERIES', 'OPEN','HIGH','LOW','CLOSE', 'TIMESTAMP', 'UM', 'DM', 'UM_SMA','DM_SMA', 'UM_EMA', 'DM_EMA', 'RS_SMA', 'RS_EMA', 'RSI_SMA', 'RSI_EMA']
    df['TIMESTAMP'] = pd.to_datetime(df['TIMESTAMP'])
    df.sort_values(by='TIMESTAMP', inplace = True)
    df.reset_index(drop = True, inplace=True)
    
 
    # Resampling based on time_interval provided
    if (time_interval == 'D'):
        df = df[cols_seq]
        df = df.iloc[-120:]
    elif (time_interval == 'W'):
        # Grouping based on required values - weekly interval
        df = group_df(df,date_col = 'TIMESTAMP', interval='W')
        df = df[cols_seq]
        # Sorting based on TIMESTAMP
        df.sort_values(by='TIMESTAMP', inplace = True)
        df.reset_index(drop=True, inplace=True)
        df = df.iloc[-120:]
    elif (time_interval == 'M'):
        df = group_df(df,date_col = 'TIMESTAMP', interval='M')
        df = df[cols_seq]
    else:
        print ('Invalid Time Interval. Please try again with options -(D/ W/ M)')
    
    
    # Calculating Upward / Downward Movement
    if (df.iat[-1,5] >= df.iat[-2,5]):
        df.iat[-1,7] = df.iat[-1,5] - df.iat[-2,5]
        df.iat[-1,8] = 0
    else:
        df.iat[-1,8] = df.iat[-2,5] - df.iat[-1,5]
        df.iat[-1,7] = 0
 
    # Using SMA
    UM_SMA = df.iloc[-period:,7].values.mean()
    DM_SMA = df.iloc[-period:,8].values.mean()
    RS_SMA = UM_SMA/DM_SMA
    RSI_SMA = 100-(100/(RS_SMA+1))
    # Using EMA
    UM_EMA = df.loc[:,'UM'].ewm(com = period-1, min_periods=period).mean().values[-1]
    DM_EMA = df.loc[:,'DM'].ewm(com = period-1, min_periods=period).mean().values[-1]
    RS_EMA = UM_EMA/DM_EMA
    RSI_EMA = 100-(100/(RS_EMA+1))
    # Other values
 
    df.iat[-1, 9] = UM_SMA
    df.iat[-1, 10] = DM_SMA
    df.iat[-1, 11] = UM_EMA
    df.iat[-1, 12] = DM_EMA
    df.iat[-1, 13] = RS_SMA
    df.iat[-1,14] = RS_EMA
    df.iat[-1,15] = RSI_SMA
    df.iat[-1,16] = RSI_EMA
 
    return df.iloc[-1:]

In [7]:
nifty_500 = pd.read_csv('drive/MyDrive/trading/ind_nifty500list.csv')

In [8]:
# Nifty 500 list
 
nifty_500_symbols = np.array(nifty_500['Symbol'])
print (len(nifty_500_symbols))
print (type(nifty_500_symbols))

501
<class 'numpy.ndarray'>


Loading Existing Master Files

In [9]:
os.listdir('drive/MyDrive/trading/NseStocks_Masterfiles')

['master_df.csv',
 'master_df_nifty500.csv',
 'master_df_RSI_nifty500.csv',
 'master_df_RSI_backup.csv',
 'master_df_RSI.csv',
 'master_monthly_RSI.csv',
 'master_weekly_RSI.csv',
 'master_weekly_ich.csv',
 'master_daily_ich.csv',
 'master_df_swing.csv']

In [10]:
df = pd.read_csv('drive/MyDrive/trading/NseStocks_Masterfiles/master_df_RSI.csv',infer_datetime_format = True, parse_dates=['TIMESTAMP'], encoding='utf-8')
#df_w = pd.read_csv('drive/MyDrive/trading/NseStocks_Masterfiles/master_weekly_RSI.csv')
#df_m = pd.read_csv('drive/MyDrive/trading/NseStocks_Masterfiles/master_monthly_RSI.csv')
 
# Converting Timestamp column to Datetime data type.
# df['TIMESTAMP'] = pd.to_datetime(df['TIMESTAMP'])
#df_w['TIMESTAMP'] = pd.to_datetime(df_w['TIMESTAMP'])
#df_m['TIMESTAMP'] = pd.to_datetime(df_m['TIMESTAMP'])

In [11]:
# Storing a backup

df.to_csv('drive/MyDrive/trading/NseStocks_Masterfiles/master_df_RSI_backup.csv',encoding='utf-8',index=False)

In [12]:
df.loc[df['SYMBOL']=='ICICIBANK'][-3:]

,SYMBOL,SERIES,OPEN,HIGH,LOW,CLOSE,TIMESTAMP,UM,DM,UM_SMA,DM_SMA,UM_EMA,DM_EMA,RS_SMA,RS_EMA,RSI_SMA,RSI_EMA
495450,ICICIBANK,EQ,622.55,628.45,609.0,621.45,2021-04-29,0.1,0.00,7.732143,4.603571,7.310656,4.549663,1.679597,1.606856,62.680950,61.639621
495951,ICICIBANK,EQ,610.10,616.35,598.1,600.50,2021-04-30,0.0,20.95,7.732143,6.032143,6.788207,5.721321,1.281824,1.186475,56.175402,54.264291
496452,ICICIBANK,EQ,592.55,598.85,587.5,596.75,2021-05-03,0.0,3.75,7.732143,5.550000,6.303335,5.580483,1.393179,1.129532,58.214574,53.041329


Downloading the latest BHAVCOPY

In [13]:
# Specify the number of days before which the BHAVCOPY is required
delta = 0

In [14]:
# To Get the Path
 
dt_ = date.today()-datetime.timedelta(days=delta)
year = dt_.strftime("%Y")
month = dt_.strftime("%b").upper()
dt = dt_.strftime("%d")
 
path_ = 'https://archives.nseindia.com/content/historical/EQUITIES/'+year+'/'+month+'/cm'+dt+month+year+'bhav.csv.zip'
print (path_)
path_local = 'drive/MyDrive/trading/bhavcopy/'+ path_[-23:-4]
# To download the File from the path
 
!curl -O $path_  
# To unzip a file in the drive
zippedFile = path_[-23:]
!unzip $zippedFile -d 'drive/MyDrive/trading/bhavcopy/'
 
# # To Remove a file from the drive
!rm $zippedFile

https://archives.nseindia.com/content/historical/EQUITIES/2021/MAY/cm04MAY2021bhav.csv.zip
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 75140  100 75140    0     0   154k      0 --:--:-- --:--:-- --:--:--  154k
Archive:  cm04MAY2021bhav.csv.zip
  inflating: drive/MyDrive/trading/bhavcopy/cm04MAY2021bhav.csv  


Concatenating the Latest Bhavcopy to the existing daily interval master file

In [15]:
 
desired_cols = ['SYMBOL', 'SERIES','OPEN','HIGH','LOW','CLOSE','TIMESTAMP']
 
df_bhav = pd.read_csv(path_local)
df_bhav = df_bhav.loc[df_bhav['SERIES']=='EQ'][desired_cols]
df_bhav['TIMESTAMP'] = pd.to_datetime(df_bhav['TIMESTAMP'])
df_bhav = df_bhav.loc[df_bhav['SYMBOL'].isin(nifty_500_symbols)]
 
#   Updating the latest data to the existing master dataframes
 
master_df = pd.concat([df,df_bhav]).reset_index(drop=True)

**Loading the latest BHAVCOPY & calculating the RSI - Daily Interval**

In [16]:
desired_cols = ['SYMBOL', 'SERIES','OPEN','HIGH','LOW','CLOSE','TIMESTAMP']
 
try:
    
    curr_day_df = pd.DataFrame(columns = ['SYMBOL', 'SERIES', 'OPEN','HIGH','LOW','CLOSE', 'TIMESTAMP', 'UM', 'DM', 'UM_SMA', 'DM_SMA', 'UM_EMA', 'DM_EMA', 'RS_SMA', 'RS_EMA', 'RSI_SMA', 'RSI_EMA'])
    for i in nifty_500_symbols:
        curr_day_df = curr_day_df.append(rsi(master_df.loc[master_df['SYMBOL']==i]))
 
    # Filtering out the last record of each stock with reference to the date
    master_df = master_df.loc[master_df['TIMESTAMP']!=master_df['TIMESTAMP'].max()]
 
    # Appending the last record updated with RSI values of each stock with reference to the date 
    master_df = master_df.append(curr_day_df)
    master_df.reset_index(drop=True, inplace=True)
    master_df.to_csv('drive/MyDrive/trading/NseStocks_Masterfiles/master_df_RSI.csv',index=False)
    print (f"Successfully added the RSI values to {path_[-23:-8]} in the master file. :-)")
 
except:
    print ("Bhavcopy for the given date is not found in the given path.")

Successfully added the RSI values to cm04MAY2021bhav in the master file. :-)


**Calculating RSI - Weekly & Monthly Interval**

In [17]:
cols_df_RSI = ['SYMBOL', 'SERIES', 'OPEN','HIGH','LOW','CLOSE', 'TIMESTAMP', 'UM', 'DM', 'UM_SMA', 'DM_SMA', 'UM_EMA', 'DM_EMA', 'RS_SMA', 'RS_EMA', 'RSI_SMA', 'RSI_EMA']
 
master_monthly_df = pd.DataFrame(columns=cols_df_RSI)
master_weekly_df = pd.DataFrame(columns=cols_df_RSI)

for symbol in nifty_500_symbols:
    try:
        temp_df = master_df.loc[master_df['SYMBOL']==symbol]
        
        
        # Grouping by Month dataframe, and Grouping by Week dataframe.
        
        # Getting year, month, & weeknum. Weeknum is common across years to we need to create unique index by using year and weeknum
        temp_df['Year'] = temp_df['TIMESTAMP'].dt.year
        # Getting month number
        temp_df['Month_Number'] = temp_df['TIMESTAMP'].dt.month
        # Getting week number
        temp_df['Week_Number'] = temp_df['TIMESTAMP'].dt.week
 
        # Grouping based on required values - weekly interval
        temp_df_w = temp_df.groupby(['Year','Week_Number']).agg({'SYMBOL':'last','SERIES':'last','OPEN':'first', 'HIGH':'max', 'LOW':'min', 'CLOSE':'last', 'TIMESTAMP':'last'})
        temp_df_w.reset_index(drop=True,inplace=True)
        
        # Grouping based on required values - monthly interval
        temp_df_m = temp_df.groupby(['Year','Month_Number']).agg({'SYMBOL':'last','SERIES':'last','OPEN':'first', 'HIGH':'max', 'LOW':'min', 'CLOSE':'last', 'TIMESTAMP':'last'})
        temp_df_m.reset_index(drop=True,inplace=True)
 
        # Monthly RSI Data
   
        temp_df_m = rsi_all(temp_df_m,period=14)
        master_monthly_df = master_monthly_df.append(temp_df_m)
        
        
        # Weekly RSI Data
 
        temp_df_w = rsi_all(temp_df_w,period=14)
        master_weekly_df = master_weekly_df.append(temp_df_w)
        
        # print (f'{symbol} processed successfully.')
    except:
        print (f'{symbol} - failed')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.


Writing the Weekly & Monthly RSI files to Drive

In [18]:
master_weekly_df.to_csv('drive/MyDrive/trading/NseStocks_Masterfiles/master_weekly_RSI.csv',encoding='utf-8',index=False)
master_monthly_df.to_csv('drive/MyDrive/trading/NseStocks_Masterfiles/master_monthly_RSI.csv',encoding='utf-8',index=False)

Loading the updated master files for testing and screening

In [19]:
df = pd.read_csv('drive/MyDrive/trading/NseStocks_Masterfiles/master_df_RSI.csv',infer_datetime_format = True, parse_dates=['TIMESTAMP'], encoding='utf-8')
df_w = pd.read_csv('drive/MyDrive/trading/NseStocks_Masterfiles/master_weekly_RSI.csv',infer_datetime_format = True, parse_dates=['TIMESTAMP'], encoding='utf-8')
df_m = pd.read_csv('drive/MyDrive/trading/NseStocks_Masterfiles/master_monthly_RSI.csv',infer_datetime_format = True, parse_dates=['TIMESTAMP'], encoding='utf-8')

In [20]:
df_m.loc[df_m['SYMBOL']=='SBIN'].tail(5)

,SYMBOL,SERIES,OPEN,HIGH,LOW,CLOSE,TIMESTAMP,UM,DM,UM_SMA,DM_SMA,UM_EMA,DM_EMA,RS_SMA,RS_EMA,RSI_SMA,RSI_EMA
20406,SBIN,EQ,274.9,310.9,269.50,282.10,2021-01-29,7.15,0.00,10.528571,14.796429,13.490019,11.322654,0.711562,1.191418,41.57,54.37
20407,SBIN,EQ,285.1,427.7,282.75,390.15,2021-02-26,108.05,0.00,18.246429,14.217857,20.414581,10.493504,1.283346,1.945449,56.20,66.05
20408,SBIN,EQ,395.1,408.9,345.20,364.30,2021-03-31,0.00,25.85,18.246429,14.971429,18.922323,11.616028,1.218750,1.628984,54.93,61.96
20409,SBIN,EQ,367.7,371.9,321.30,353.50,2021-04-30,0.00,10.80,18.246429,14.639286,17.541450,11.556478,1.246402,1.517889,55.48,60.28
20410,SBIN,EQ,349.6,362.9,341.40,351.50,2021-05-04,0.00,2.00,18.246429,7.200000,16.263325,10.860163,2.534226,1.497521,71.71,59.96


In [21]:
df_w.loc[df_w['SYMBOL']=='SBIN'].tail(10)

,SYMBOL,SERIES,OPEN,HIGH,LOW,CLOSE,TIMESTAMP,UM,DM,UM_SMA,DM_SMA,UM_EMA,DM_EMA,RS_SMA,RS_EMA,RSI_SMA,RSI_EMA
87619,SBIN,EQ,395.10,408.90,378.10,383.65,2021-03-05,0.00,6.50,11.671429,3.089286,10.598833,3.539801,3.778035,2.994189,79.07,74.96
87620,SBIN,EQ,388.45,394.90,377.50,381.10,2021-03-12,0.00,2.55,11.032143,3.271429,9.841774,3.469101,3.372271,2.836981,77.13,73.94
87621,SBIN,EQ,382.50,387.85,357.25,371.15,2021-03-19,0.00,9.95,11.032143,3.910714,9.138790,3.932022,2.821005,2.324196,73.83,69.92
87622,SBIN,EQ,372.00,377.95,345.20,357.20,2021-03-26,0.00,13.95,11.032143,4.578571,8.486019,4.647592,2.409516,1.825896,70.67,64.61
87623,SBIN,EQ,360.10,371.90,356.30,370.65,2021-04-01,13.45,0.00,11.414286,4.578571,8.840589,4.315621,2.492980,2.048509,71.37,67.20
87624,SBIN,EQ,367.50,369.20,347.60,353.00,2021-04-09,0.00,17.65,11.096429,5.839286,8.209118,5.268077,1.900306,1.558276,65.52,60.91
87625,SBIN,EQ,344.00,347.55,322.55,339.90,2021-04-16,0.00,13.10,10.625000,6.775000,7.622753,5.827500,1.568266,1.308066,61.06,56.67
87626,SBIN,EQ,327.15,339.90,321.30,336.45,2021-04-23,0.00,3.45,9.350000,7.021429,7.078270,5.657679,1.331638,1.251091,57.11,55.58
87627,SBIN,EQ,339.25,369.95,339.25,353.50,2021-04-30,17.05,0.00,10.567857,5.582143,7.790537,5.253559,1.893154,1.482907,65.44,59.72
87628,SBIN,EQ,349.60,362.90,341.40,351.50,2021-05-04,0.00,2.00,10.567857,5.610714,7.234070,5.021162,1.883514,1.440716,65.32,59.03


# ICHIMOKU values

In [22]:
def ichimoku(df):
    # Tenken - ( HH + LL ) / 2 - 9 Periods
    df = df.iloc[-100:]
    cols_ls = ['SYMBOL', 'SERIES', 'OPEN', 'HIGH', 'LOW', 'CLOSE', 'TIMESTAMP','TENKEN_9','KIJUN_26','SENKOU_A','SENKOU_B_52']
    df.sort_values(by='TIMESTAMP',inplace=True)
    df.reset_index(drop=True,inplace=True)
    df['TENKEN_9_HH'] = df['HIGH'].rolling(9).max()
    df['TENKEN_9_LL'] = df['LOW'].rolling(9).min()
    df['TENKEN_9'] = (df['TENKEN_9_HH']+df['TENKEN_9_LL'])/2
    
    # Kijun  - (HH + LL) / 2  - 26 Periods
    df['KIJUN_26_HH'] = df['HIGH'].rolling(26).max()
    df['KIJUN_26_LL'] = df['LOW'].rolling(26).min()
    df['KIJUN_26'] = (df['KIJUN_26_HH']+df['KIJUN_26_LL'])/2
    # Senkou A  - ( Tenken + Kijun ) / 2
    df['SENKOU_A'] = (df['TENKEN_9']+df['KIJUN_26'])/2
    # Senkou B - (HH + LL) /2 - 52 Periods
    df['SENKOU_B_52_HH'] = df['HIGH'].rolling(52).max()
    df['SENKOU_B_52_LL'] = df['LOW'].rolling(52).min()
    df['SENKOU_B_52'] = (df['SENKOU_B_52_HH']+df['SENKOU_B_52_LL'])/2
    df = df[cols_ls]
    return df

**Applying Ichimoku Function to all the scripts of the dataframe**

ICHIMOKU calculation - Daily Interval

In [23]:
cols_ls = ['SYMBOL', 'SERIES', 'OPEN', 'HIGH', 'LOW', 'CLOSE', 'TIMESTAMP','TENKEN_9','KIJUN_26','SENKOU_A','SENKOU_B_52']
df_ichimoku_d = pd.DataFrame(columns = cols_ls)
 
for i in nifty_500_symbols:
    try:
        temp_ich_d = df.loc[df['SYMBOL']==i][['SYMBOL', 'SERIES', 'OPEN', 'HIGH', 'LOW', 'CLOSE', 'TIMESTAMP']]
        temp_ich_d['TIMESTAMP'] = pd.to_datetime(temp_ich_d['TIMESTAMP'])
        temp_ich_d.sort_values(by='TIMESTAMP', inplace=True)
        df_ichimoku_d = df_ichimoku_d.append(ichimoku(temp_ich_d))
    except:
        print(f'Failed for {i}')

ICHIMOKU calculation - Weekly Interval

In [24]:
cols_ls = ['SYMBOL', 'SERIES', 'OPEN', 'HIGH', 'LOW', 'CLOSE', 'TIMESTAMP','TENKEN_9','KIJUN_26','SENKOU_A','SENKOU_B_52']
 
df_ichimoku_w = pd.DataFrame(columns = cols_ls)
for i in nifty_500_symbols:
    try:
        temp_ich_w = df_w.loc[df_w['SYMBOL']==i][['SYMBOL', 'SERIES', 'OPEN', 'HIGH', 'LOW', 'CLOSE', 'TIMESTAMP']]
        temp_ich_w['TIMESTAMP'] = pd.to_datetime(temp_ich_w['TIMESTAMP'])
        temp_ich_w.sort_values(by='TIMESTAMP', inplace=True)
        df_ichimoku_w = df_ichimoku_w.append(ichimoku(temp_ich_w))
    except:
        print(f'Failed for {i}')

In [25]:
df_ichimoku_d.loc[df_ichimoku_d['SYMBOL']=='ICICIBANK'][-10:]

,SYMBOL,SERIES,OPEN,HIGH,LOW,CLOSE,TIMESTAMP,TENKEN_9,KIJUN_26,SENKOU_A,SENKOU_B_52
90,ICICIBANK,EQ,565.20,572.65,552.60,559.10,2021-04-20,558.500,584.575,571.5375,605.275
91,ICICIBANK,EQ,552.00,580.60,546.90,579.20,2021-04-22,558.500,584.575,571.5375,605.275
92,ICICIBANK,EQ,569.90,580.50,567.05,569.95,2021-04-23,555.875,572.000,563.9375,605.275
93,ICICIBANK,EQ,602.00,605.50,588.00,591.10,2021-04-26,568.325,569.075,568.7000,605.275
94,ICICIBANK,EQ,593.25,601.95,591.10,598.75,2021-04-27,569.950,568.325,569.1375,605.275
95,ICICIBANK,EQ,598.00,622.80,598.00,621.35,2021-04-28,578.600,576.975,577.7875,605.275
96,ICICIBANK,EQ,622.55,628.45,609.00,621.45,2021-04-29,581.425,579.800,580.6125,605.275
97,ICICIBANK,EQ,610.10,616.35,598.10,600.50,2021-04-30,581.425,579.800,580.6125,605.275
98,ICICIBANK,EQ,592.55,598.85,587.50,596.75,2021-05-03,587.675,579.800,583.7375,605.275
99,ICICIBANK,EQ,596.65,608.00,589.00,591.55,2021-05-04,587.675,579.800,583.7375,605.275


In [26]:
df_ichimoku_w.loc[df_ichimoku_w['SYMBOL']=='ICICIBANK'][-10:]

,SYMBOL,SERIES,OPEN,HIGH,LOW,CLOSE,TIMESTAMP,TENKEN_9,KIJUN_26,SENKOU_A,SENKOU_B_52
90,ICICIBANK,EQ,607.55,633.75,599.00,609.45,2021-03-05,595.700,506.575,551.1375,473.850
91,ICICIBANK,EQ,611.60,638.00,601.15,612.85,2021-03-12,595.700,506.575,551.1375,473.850
92,ICICIBANK,EQ,611.90,612.85,570.00,586.65,2021-03-19,595.700,514.250,554.9750,480.450
93,ICICIBANK,EQ,586.85,589.25,557.10,578.55,2021-03-26,595.700,525.075,560.3875,482.475
94,ICICIBANK,EQ,582.00,596.10,579.00,594.40,2021-04-01,615.200,533.725,574.4625,482.475
95,ICICIBANK,EQ,592.00,592.00,559.40,566.20,2021-04-09,618.250,533.725,575.9875,482.475
96,ICICIBANK,EQ,551.40,577.75,531.15,566.95,2021-04-16,605.275,533.725,569.5000,482.475
97,ICICIBANK,EQ,550.00,580.60,534.40,569.95,2021-04-23,589.775,543.200,566.4875,482.475
98,ICICIBANK,EQ,602.00,628.45,588.00,600.50,2021-04-30,584.575,564.200,574.3875,482.475
99,ICICIBANK,EQ,592.55,608.00,587.50,591.55,2021-05-04,584.575,572.600,578.5875,482.475


Writing the dataframes to master files

In [27]:
df_ichimoku_w.to_csv('drive/MyDrive/trading/NseStocks_Masterfiles/master_weekly_ich.csv',encoding='utf-8',index=False)
df_ichimoku_d.to_csv('drive/MyDrive/trading/NseStocks_Masterfiles/master_daily_ich.csv',encoding='utf-8',index=False)

In [28]:
# df_ichimoku_w= pd.read_csv('drive/MyDrive/trading/NseStocks_Masterfiles/master_weekly_ich.csv',encoding='utf-8')
# df_ichimoku_d = pd.read_csv('drive/MyDrive/trading/NseStocks_Masterfiles/master_daily_ich.csv',encoding='utf-8')

In [29]:
df_ichimoku_d.columns

Index(['SYMBOL', 'SERIES', 'OPEN', 'HIGH', 'LOW', 'CLOSE', 'TIMESTAMP',
       'TENKEN_9', 'KIJUN_26', 'SENKOU_A', 'SENKOU_B_52'],
      dtype='object')

In [30]:
df_ichimoku_d.loc[df_ichimoku_d['SYMBOL'] =='BANKINDIA'].tail()

,SYMBOL,SERIES,OPEN,HIGH,LOW,CLOSE,TIMESTAMP,TENKEN_9,KIJUN_26,SENKOU_A,SENKOU_B_52
95,BANKINDIA,EQ,66.8,68.50,66.50,68.15,2021-04-28,67.425,71.70,69.5625,79.375
96,BANKINDIA,EQ,68.6,69.20,66.65,67.50,2021-04-29,67.425,71.70,69.5625,79.975
97,BANKINDIA,EQ,67.0,67.80,65.00,66.35,2021-04-30,65.900,71.75,68.8250,79.975
98,BANKINDIA,EQ,65.0,66.35,64.60,65.55,2021-05-03,65.900,71.75,68.8250,79.975
99,BANKINDIA,EQ,66.2,71.65,66.10,68.55,2021-05-04,67.125,71.75,69.4375,79.975


# Swing High & Swing Low calculation

Utility Function to compare the columns and extract Higher Highs & Lower Lows

In [31]:
def SwingHigh(row):
    if (row['HH_bw_1'] > 0) and (row['HH_bw_2']>0) and (row['HH_fw_1']>0) and (row['HH_fw_2']>0):
        return row['HIGH']
    else:
        return np.nan

def SwingLow(row):
    if (row['LL_bw_1'] < 0) and (row['LL_bw_2']<0) and (row['LL_fw_1']<0) and (row['LL_fw_2']<0):
        return row['LOW']
    else:
        return np.nan

In [32]:
df = pd.read_csv('drive/MyDrive/trading/NseStocks_Masterfiles/master_df_RSI.csv',infer_datetime_format = True, parse_dates=['TIMESTAMP'], encoding='utf-8')

In [33]:
# df['TIMESTAMP'] = pd.to_datetime(df['TIMESTAMP'])
df = df[['SYMBOL', 'SERIES', 'OPEN', 'HIGH', 'LOW', 'CLOSE', 'TIMESTAMP']]

In [34]:
df.columns

Index(['SYMBOL', 'SERIES', 'OPEN', 'HIGH', 'LOW', 'CLOSE', 'TIMESTAMP'], dtype='object')

In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 497242 entries, 0 to 497241
Data columns (total 7 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   SYMBOL     497242 non-null  object        
 1   SERIES     497242 non-null  object        
 2   OPEN       497242 non-null  float64       
 3   HIGH       497242 non-null  float64       
 4   LOW        497242 non-null  float64       
 5   CLOSE      497242 non-null  float64       
 6   TIMESTAMP  497242 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(4), object(2)
memory usage: 26.6+ MB


In [36]:
start_date = '2019-01-01'

In [37]:
df = df.loc[df['TIMESTAMP']>=start_date]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 282139 entries, 490 to 497241
Data columns (total 7 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   SYMBOL     282139 non-null  object        
 1   SERIES     282139 non-null  object        
 2   OPEN       282139 non-null  float64       
 3   HIGH       282139 non-null  float64       
 4   LOW        282139 non-null  float64       
 5   CLOSE      282139 non-null  float64       
 6   TIMESTAMP  282139 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(4), object(2)
memory usage: 17.2+ MB


In [38]:
df_swing_cols = ['SYMBOL', 'SERIES', 'OPEN', 'HIGH', 'LOW', 'CLOSE', 'TIMESTAMP', 'swing_high', 'swing_low']
df_swing = pd.DataFrame(columns=df_swing_cols)

for i in nifty_500_symbols:
    try:
        df_t = df.loc[df['SYMBOL']==i]
        df_t.sort_values(by='TIMESTAMP',inplace=True)
        df_t.reset_index(drop=True,inplace=True)
        df_t['HH_bw_1'] = df_t['HIGH'].diff(periods=1)
        df_t['HH_bw_2'] = df_t['HIGH'].diff(periods=2)
        df_t['HH_fw_1'] = df_t['HIGH'].diff(periods=-1)
        df_t['HH_fw_2'] = df_t['HIGH'].diff(periods=-2)
        df_t['LL_bw_1'] = df_t['LOW'].diff(periods=1)
        df_t['LL_bw_2'] = df_t['LOW'].diff(periods=2)
        df_t['LL_fw_1'] = df_t['LOW'].diff(periods=-1)
        df_t['LL_fw_2'] = df_t['LOW'].diff(periods=-2)
        df_t['swing_high'] = df_t.apply(lambda row :SwingHigh(row), axis = 1)
        df_t['swing_low'] = df_t.apply(lambda row :SwingLow(row), axis = 1)
        df_t = df_t[df_swing_cols]
        df_t.fillna(method='ffill', inplace=True)
        df_swing = df_swing.append(df_t)
    except:
        print(f'Failed for {i}')

df_swing.reset_index(drop=True,inplace=True)

In [39]:
df_swing.tail(10)

,SYMBOL,SERIES,OPEN,HIGH,LOW,CLOSE,TIMESTAMP,swing_high,swing_low
282129,ECLERX,EQ,1208.00,1245.00,1190.05,1222.45,2021-04-20,1248.0,1110.0
282130,ECLERX,EQ,1215.00,1220.00,1181.20,1207.05,2021-04-22,1248.0,1110.0
282131,ECLERX,EQ,1196.10,1271.00,1180.25,1195.70,2021-04-23,1271.0,1110.0
282132,ECLERX,EQ,1195.00,1213.35,1162.90,1171.25,2021-04-26,1271.0,1110.0
282133,ECLERX,EQ,1171.00,1194.50,1160.00,1165.65,2021-04-27,1271.0,1160.0
282134,ECLERX,EQ,1168.00,1207.00,1168.00,1177.80,2021-04-28,1271.0,1160.0
282135,ECLERX,EQ,1188.75,1248.00,1173.00,1230.60,2021-04-29,1271.0,1160.0
282136,ECLERX,EQ,1212.55,1266.05,1209.00,1245.85,2021-04-30,1271.0,1160.0
282137,ECLERX,EQ,1244.90,1275.00,1237.35,1266.50,2021-05-03,1271.0,1160.0
282138,ECLERX,EQ,1273.75,1293.00,1234.90,1241.70,2021-05-04,1271.0,1160.0


In [40]:
df_swing.loc[df_swing['SYMBOL']=='SBIN'][-10:]

,SYMBOL,SERIES,OPEN,HIGH,LOW,CLOSE,TIMESTAMP,swing_high,swing_low
233160,SBIN,EQ,335.10,339.90,326.70,329.50,2021-04-20,347.55,321.3
233161,SBIN,EQ,326.00,337.65,323.30,336.65,2021-04-22,347.55,321.3
233162,SBIN,EQ,331.65,339.90,331.65,336.45,2021-04-23,347.55,321.3
233163,SBIN,EQ,339.25,347.45,339.25,344.30,2021-04-26,347.55,321.3
233164,SBIN,EQ,344.00,354.95,342.40,353.05,2021-04-27,347.55,321.3
233165,SBIN,EQ,357.00,364.30,356.05,363.40,2021-04-28,347.55,321.3
233166,SBIN,EQ,365.00,369.95,355.50,359.40,2021-04-29,369.95,321.3
233167,SBIN,EQ,353.45,362.50,350.45,353.50,2021-04-30,369.95,321.3
233168,SBIN,EQ,349.60,352.50,341.40,350.60,2021-05-03,369.95,321.3
233169,SBIN,EQ,355.00,362.90,350.20,351.50,2021-05-04,369.95,321.3


In [41]:
df_swing.to_csv('drive/MyDrive/trading/NseStocks_Masterfiles/master_df_swing.csv',encoding='utf-8',index=False)